In [90]:
import pandas as pd


In [91]:
file_path = '/content/Canada_Hosp1_COVID_InpatientData.xlsx'
xls = pd.ExcelFile(file_path)

days_breakdown = pd.read_excel(xls, sheet_name='Days-breakdown')
data_at_admission = pd.read_excel(xls, sheet_name='Data-at-admission')
hlos = pd.read_excel(xls, sheet_name='Hospital-length-of-stay')

merge_data = pd.merge(days_breakdown, data_at_admission[['id', 'age', 'sex', 'height', 'weight', 'comorbidities']], left_on='parent_id', right_on='id', how='inner')

merge_data = pd.merge(merge_data, hlos[['parent_id', 'hospital_length_of_stay']], on='parent_id', how='inner')

# Create the adjusted HLOS (HLOS - day)
merge_data['adjusted_hlos'] = merge_data['hospital_length_of_stay'] - merge_data['day']

# Drop rows where adjusted HLOS is less than or equal to zero
processed_data = merge_data[merge_data['adjusted_hlos'] > 0]

# Select the required features
req_features = [
    'age', 'sex', 'height', 'weight', 'comorbidities',
    'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate',
    'respiratory_rate', 'oxygen_saturation', 'temperature',
    'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw',
    'platelet_count', 'aptt_aptr', 'pt', 'alt', 'ast', 'serum_creatinine',
    'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2', 'pao2_fio2',
    'ph', 'ferritin', 'd_dimer', 'crp', 'hs_crp', 'intubated',
    'adjusted_hlos'  # The target variable
]

process_data = processed_data[req_features]
process_data.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,aptt_aptr,pt,alt,ast,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,pao2,pao2_fio2,ph,ferritin,d_dimer,crp,hs_crp,intubated,adjusted_hlos
0,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",119.0,54.0,79.0,18.0,94.0,37.2,8.4,4.97,116.0,0.354,71.2,23.3,328.0,18.0,116.0,44.0,NaN,NaN,NaN,102.0,142.0,4.1,NaN,NaN,NaN,NaN,NaN,921.0,NaN,NaN,NaN,No,20
1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",133.0,64.0,73.0,18.0,98.0,37.1,5.2,4.99,115.0,0.359,71.9,23.0,320.0,18.4,117.0,NaN,NaN,NaN,NaN,88.0,144.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,19
2,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",140.0,74.0,70.0,20.0,95.0,37.5,2.5,5.26,121.0,0.372,70.7,23.0,325.0,18.6,145.0,NaN,NaN,NaN,NaN,73.0,142.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,18
3,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",154.0,78.0,77.0,18.0,95.0,37.1,6.9,5.38,124.0,0.383,71.2,23.0,324.0,18.5,218.0,NaN,NaN,NaN,NaN,85.0,141.0,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,17
4,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",155.0,61.0,64.0,16.0,92.0,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,16


In [92]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2895 entries, 0 to 4059
Data columns (total 78 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id_x                                2895 non-null   int64  
 1   parent_id                           2895 non-null   int64  
 2   day                                 2895 non-null   int64  
 3   systolic_blood_pressure             2872 non-null   float64
 4   diastolic_blood_pressure            2872 non-null   float64
 5   heart_rate                          2888 non-null   float64
 6   respiratory_rate                    2887 non-null   float64
 7   oxygen_saturation                   2887 non-null   float64
 8   temperature                         2888 non-null   float64
 9   highest_mean_arterial_pressure      759 non-null    float64
 10  lowest_mean_arterial_pressure       759 non-null    float64
 11  highest_heart_rate                  2888 non-nul

In [93]:
merge_data

,id_x,parent_id,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,lowest_mean_arterial_pressure,highest_heart_rate,lowest_heart_rate,highest_creatinine,lowest_urine_output,highest_gcs,lowest_gcs,intubated,highest_ph,lowest_ph,cardiac_arrest,arrested_time,major_cardiac_events,clinically_diagnosed_infections,mechanical_ventilation,number_of_vasoactive_agents,antiarrhythmic_therapies,renal_replacement_therapy_dialysis,cardiovascular_mechanical_support,fluid_balance,wbc,rbc,hemoglobin,hematocrit,platelet_count,aptt_aptr,pt,alt,ast,mch,mcv,mchc,rdw,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,pao2,pao2_fio2,ph,high_sensitivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,echocardiogram,ejection_fraction,wall_motion_abnormality,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,id_y,age,sex,height,weight,comorbidities,hospital_length_of_stay,adjusted_hlos
0,1,1,1,119.0,54.0,79.0,18.0,94.0,37.2,85.0,85.0,80.0,73.0,102.0,700.0,15.0,15.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-700.0,8.4,4.97,116.0,0.354,116.0,44.0,NaN,NaN,NaN,23.3,71.2,328.0,18.0,102.0,142.0,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,921.0,NaN,NaN,NaN,No,NaN,NaN,No,"""[\""Unilateral Consolidation\""]""",No,"""[\""Bilateral consolidation/infiltration\""]""",No,"""[]""",Yes,Yes,No,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,20
1,2,1,2,133.0,64.0,73.0,18.0,98.0,37.1,NaN,NaN,73.0,69.0,88.0,1351.0,14.0,13.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-1301.0,5.2,4.99,115.0,0.359,117.0,NaN,NaN,NaN,NaN,23.0,71.9,320.0,18.4,88.0,144.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,19
2,3,1,3,140.0,74.0,70.0,20.0,95.0,37.5,NaN,NaN,72.0,65.0,73.0,1420.0,13.0,13.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-1420.0,2.5,5.26,121.0,0.372,145.0,NaN,NaN,NaN,NaN,23.0,70.7,325.0,18.6,73.0,142.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,18
3,4,1,4,154.0,78.0,77.0,18.0,95.0,37.1,NaN,NaN,80.0,72.0,85.0,350.0,15.0,14.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-350.0,6.9,5.38,124.0,0.383,218.0,NaN,NaN,NaN,NaN,23.0,71.2,324.0,18.5,85.0,141.0,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Yes,"""[\""Unilateral Consolidation\""]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,17
4,5,1,5,155.0,61.0,64.0,16.0,92.0,36.7,NaN,NaN,77.0,64.0,NaN,NaN,10.0,10.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4059,4260,522,4,98.0,73.0,82.0,24.0,95.0,36.1,NaN,NaN,82.0,75.0,NaN,NaN,15.0,15.0,No,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,No,522,73,Male,NaN,NaN,"""[]""",5,1
4060,4261,522,5,110.0,64.0,68.0,18.0,95.0,36.5,NaN,NaN,75.0,68.0,NaN,NaN,15.0,15.0,No,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]

In [94]:
import pandas as pd
import numpy as np


# Step 1: Dropping columns with less than 30% non-missing values
threshold = 0.3 * len(processed_data)
columns_to_drop = processed_data.columns[processed_data.isnull().sum() > threshold]


clean_data = processed_data.drop(columns=columns_to_drop)

# Step 2: Handle missing values for the remaining columns
# For numerical columns, fill with the median
num_cols = clean_data.select_dtypes(include=[np.number]).columns
clean_data[num_cols] = clean_data[num_cols].apply(lambda col: col.fillna(col.median()), axis=0)

# For categorical columns, fill with the most frequent value
cat_cols = clean_data.select_dtypes(include=[object]).columns
clean_data[cat_cols] = clean_data[cat_cols].apply(lambda col: col.fillna(col.mode()[0]), axis=0)

# Display the cleaned dataset
clean_data.head()


,id_x,parent_id,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_heart_rate,lowest_heart_rate,highest_gcs,lowest_gcs,intubated,cardiac_arrest,major_cardiac_events,clinically_diagnosed_infections,mechanical_ventilation,antiarrhythmic_therapies,renal_replacement_therapy_dialysis,cardiovascular_mechanical_support,echocardiogram,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,id_y,age,sex,weight,comorbidities,hospital_length_of_stay,adjusted_hlos
0,1,1,1,119.0,54.0,79.0,18.0,94.0,37.2,80.0,73.0,15.0,15.0,No,No,No,Yes,No,No,No,No,No,No,"""[\""Unilateral Consolidation\""]""",No,"""[\""Bilateral consolidation/infiltration\""]""",No,"""[]""",Yes,Yes,No,1,74,Male,77.0,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,20
1,2,1,2,133.0,64.0,73.0,18.0,98.0,37.1,73.0,69.0,14.0,13.0,No,No,No,Yes,No,No,No,No,No,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,77.0,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,19
2,3,1,3,140.0,74.0,70.0,20.0,95.0,37.5,72.0,65.0,13.0,13.0,No,No,No,Yes,No,No,No,No,No,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,77.0,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,18
3,4,1,4,154.0,78.0,77.0,18.0,95.0,37.1,80.0,72.0,15.0,14.0,No,No,No,Yes,No,No,No,No,No,Yes,"""[\""Unilateral Consolidation\""]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,77.0,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,17
4,5,1,5,155.0,61.0,64.0,16.0,92.0,36.7,77.0,64.0,10.0,10.0,No,No,No,Yes,No,No,No,No,No,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,77.0,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,16


In [95]:
pd.set_option('display.max_columns', None)

In [96]:
merge_data

,id_x,parent_id,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,lowest_mean_arterial_pressure,highest_heart_rate,lowest_heart_rate,highest_creatinine,lowest_urine_output,highest_gcs,lowest_gcs,intubated,highest_ph,lowest_ph,cardiac_arrest,arrested_time,major_cardiac_events,clinically_diagnosed_infections,mechanical_ventilation,number_of_vasoactive_agents,antiarrhythmic_therapies,renal_replacement_therapy_dialysis,cardiovascular_mechanical_support,fluid_balance,wbc,rbc,hemoglobin,hematocrit,platelet_count,aptt_aptr,pt,alt,ast,mch,mcv,mchc,rdw,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,pao2,pao2_fio2,ph,high_sensitivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,echocardiogram,ejection_fraction,wall_motion_abnormality,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,id_y,age,sex,height,weight,comorbidities,hospital_length_of_stay,adjusted_hlos
0,1,1,1,119.0,54.0,79.0,18.0,94.0,37.2,85.0,85.0,80.0,73.0,102.0,700.0,15.0,15.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-700.0,8.4,4.97,116.0,0.354,116.0,44.0,NaN,NaN,NaN,23.3,71.2,328.0,18.0,102.0,142.0,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,921.0,NaN,NaN,NaN,No,NaN,NaN,No,"""[\""Unilateral Consolidation\""]""",No,"""[\""Bilateral consolidation/infiltration\""]""",No,"""[]""",Yes,Yes,No,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,20
1,2,1,2,133.0,64.0,73.0,18.0,98.0,37.1,NaN,NaN,73.0,69.0,88.0,1351.0,14.0,13.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-1301.0,5.2,4.99,115.0,0.359,117.0,NaN,NaN,NaN,NaN,23.0,71.9,320.0,18.4,88.0,144.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,19
2,3,1,3,140.0,74.0,70.0,20.0,95.0,37.5,NaN,NaN,72.0,65.0,73.0,1420.0,13.0,13.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-1420.0,2.5,5.26,121.0,0.372,145.0,NaN,NaN,NaN,NaN,23.0,70.7,325.0,18.6,73.0,142.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,18
3,4,1,4,154.0,78.0,77.0,18.0,95.0,37.1,NaN,NaN,80.0,72.0,85.0,350.0,15.0,14.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,-350.0,6.9,5.38,124.0,0.383,218.0,NaN,NaN,NaN,NaN,23.0,71.2,324.0,18.5,85.0,141.0,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Yes,"""[\""Unilateral Consolidation\""]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,17
4,5,1,5,155.0,61.0,64.0,16.0,92.0,36.7,NaN,NaN,77.0,64.0,NaN,NaN,10.0,10.0,No,NaN,NaN,No,NaN,No,Yes,No,NaN,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",21,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4059,4260,522,4,98.0,73.0,82.0,24.0,95.0,36.1,NaN,NaN,82.0,75.0,NaN,NaN,15.0,15.0,No,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,No,522,73,Male,NaN,NaN,"""[]""",5,1
4060,4261,522,5,110.0,64.0,68.0,18.0,95.0,36.5,NaN,NaN,75.0,68.0,NaN,NaN,15.0,15.0,No,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,"""[]

In [97]:
required_features = [
    'age', 'sex', 'height', 'weight', 'comorbidities',
    'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate',
    'respiratory_rate', 'oxygen_saturation', 'temperature',
    'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw',
    'platelet_count', 'aptt_aptr', 'pt', 'alt', 'ast', 'serum_creatinine',
    'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2', 'pao2_fio2',
    'ph', 'ferritin', 'd_dimer', 'crp', 'hs_crp', 'intubated',
    'adjusted_hlos'  # The target variable
]


In [98]:
merge_data = merge_data[merge_data['adjusted_hlos']>=0]

In [99]:
df_hospital = merge_data[req_features]

In [100]:
df_hospital.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,aptt_aptr,pt,alt,ast,serum_creatinine,sodium,potassium,total_serum_bilirubin,lactate,pao2,pao2_fio2,ph,ferritin,d_dimer,crp,hs_crp,intubated,adjusted_hlos
0,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",119.0,54.0,79.0,18.0,94.0,37.2,8.4,4.97,116.0,0.354,71.2,23.3,328.0,18.0,116.0,44.0,NaN,NaN,NaN,102.0,142.0,4.1,NaN,NaN,NaN,NaN,NaN,921.0,NaN,NaN,NaN,No,20
1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",133.0,64.0,73.0,18.0,98.0,37.1,5.2,4.99,115.0,0.359,71.9,23.0,320.0,18.4,117.0,NaN,NaN,NaN,NaN,88.0,144.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,19
2,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",140.0,74.0,70.0,20.0,95.0,37.5,2.5,5.26,121.0,0.372,70.7,23.0,325.0,18.6,145.0,NaN,NaN,NaN,NaN,73.0,142.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,18
3,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",154.0,78.0,77.0,18.0,95.0,37.1,6.9,5.38,124.0,0.383,71.2,23.0,324.0,18.5,218.0,NaN,NaN,NaN,NaN,85.0,141.0,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,17
4,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",155.0,61.0,64.0,16.0,92.0,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,16


In [101]:
df_hospital.isnull().sum()

,0
age,0
sex,0
height,1333
weight,807
comorbidities,0
systolic_blood_pressure,33
diastolic_blood_pressure,33
heart_rate,16
respiratory_rate,18
oxygen_saturation,16


In [102]:
df_hospital.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3121 entries, 0 to 4060
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       3121 non-null   int64  
 1   sex                       3121 non-null   object 
 2   height                    1788 non-null   float64
 3   weight                    2314 non-null   float64
 4   comorbidities             3121 non-null   object 
 5   systolic_blood_pressure   3088 non-null   float64
 6   diastolic_blood_pressure  3088 non-null   float64
 7   heart_rate                3105 non-null   float64
 8   respiratory_rate          3103 non-null   float64
 9   oxygen_saturation         3105 non-null   float64
 10  temperature               3101 non-null   float64
 11  wbc                       1742 non-null   float64
 12  rbc                       1742 non-null   float64
 13  hemoglobin                1742 non-null   float64
 14  hematocrit   

In [103]:
df_hospital['d_dimer'] = pd.to_numeric(df_hospital['d_dimer'], errors='coerce')

<ipython-input-103-d358bf4ac05e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hospital['d_dimer'] = pd.to_numeric(df_hospital['d_dimer'], errors='coerce')


In [104]:
df_hospital.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3121 entries, 0 to 4060
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       3121 non-null   int64  
 1   sex                       3121 non-null   object 
 2   height                    1788 non-null   float64
 3   weight                    2314 non-null   float64
 4   comorbidities             3121 non-null   object 
 5   systolic_blood_pressure   3088 non-null   float64
 6   diastolic_blood_pressure  3088 non-null   float64
 7   heart_rate                3105 non-null   float64
 8   respiratory_rate          3103 non-null   float64
 9   oxygen_saturation         3105 non-null   float64
 10  temperature               3101 non-null   float64
 11  wbc                       1742 non-null   float64
 12  rbc                       1742 non-null   float64
 13  hemoglobin                1742 non-null   float64
 14  hematocrit   

In [105]:
df = df_hospital
threshold = 0.3 * 3121
df = df_hospital.dropna(thresh=threshold, axis=1)
df.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,serum_creatinine,sodium,potassium,intubated,adjusted_hlos
0,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",119.0,54.0,79.0,18.0,94.0,37.2,8.4,4.97,116.0,0.354,71.2,23.3,328.0,18.0,116.0,102.0,142.0,4.1,No,20
1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",133.0,64.0,73.0,18.0,98.0,37.1,5.2,4.99,115.0,0.359,71.9,23.0,320.0,18.4,117.0,88.0,144.0,3.7,No,19
2,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",140.0,74.0,70.0,20.0,95.0,37.5,2.5,5.26,121.0,0.372,70.7,23.0,325.0,18.6,145.0,73.0,142.0,4.0,No,18
3,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",154.0,78.0,77.0,18.0,95.0,37.1,6.9,5.38,124.0,0.383,71.2,23.0,324.0,18.5,218.0,85.0,141.0,3.9,No,17
4,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",155.0,61.0,64.0,16.0,92.0,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,16


In [106]:
df.isnull().sum()

,0
age,0
sex,0
height,1333
weight,807
comorbidities,0
systolic_blood_pressure,33
diastolic_blood_pressure,33
heart_rate,16
respiratory_rate,18
oxygen_saturation,16


In [107]:

df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})

df['intubated'] = df['intubated'].map({'Yes': 1, 'No': 0})

df[['sex', 'intubated']].head()

<ipython-input-107-e9d715306cc4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})
<ipython-input-107-e9d715306cc4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['intubated'] = df['intubated'].map({'Yes': 1, 'No': 0})


,sex,intubated
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [108]:
df.to_excel("/content/RNN_before_cleaning.xlsx",index=False)

In [109]:
import pandas as pd
data = df
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Fill missing values in numerical columns with the median
data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())


for col in categorical_cols:
    data[col] = data[col].fillna(data[col].mode()[0])

# Check if all missing values have been handled
print(data.isnull().sum())

age                         0
sex                         0
height                      0
weight                      0
comorbidities               0
systolic_blood_pressure     0
diastolic_blood_pressure    0
heart_rate                  0
respiratory_rate            0
oxygen_saturation           0
temperature                 0
wbc                         0
rbc                         0
hemoglobin                  0
hematocrit                  0
mcv                         0
mch                         0
mchc                        0
rdw                         0
platelet_count              0
serum_creatinine            0
sodium                      0
potassium                   0
intubated                   0
adjusted_hlos               0
dtype: int64


<ipython-input-109-1a9b7cd3bff1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numerical_cols] = data[numerical_cols].fillna(data[numerical_cols].median())
<ipython-input-109-1a9b7cd3bff1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].fillna(data[col].mode()[0])


In [110]:
data.nunique()

,0
age,76
sex,2
height,69
weight,203
comorbidities,137
systolic_blood_pressure,124
diastolic_blood_pressure,88
heart_rate,98
respiratory_rate,41
oxygen_saturation,31


In [111]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
categorical_col = ['comorbidities']
for col in categorical_col:
    data[col] = data[col].astype(str)

    ledr = LabelEncoder()
    data[col] = ledr.fit_transform(data[col])
    label_encoders[col] = ledr
data.head()

<ipython-input-111-0dbd27c2d28b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].astype(str)
<ipython-input-111-0dbd27c2d28b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = ledr.fit_transform(data[col])


,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,serum_creatinine,sodium,potassium,intubated,adjusted_hlos
0,74,1,167.6,77.0,103,119.0,54.0,79.0,18.0,94.0,37.2,8.4,4.97,116.0,0.3540,71.2,23.3,328.0,18.0,116.0,102.0,142.0,4.1,0,20
1,74,1,167.6,77.0,103,133.0,64.0,73.0,18.0,98.0,37.1,5.2,4.99,115.0,0.3590,71.9,23.0,320.0,18.4,117.0,88.0,144.0,3.7,0,19
2,74,1,167.6,77.0,103,140.0,74.0,70.0,20.0,95.0,37.5,2.5,5.26,121.0,0.3720,70.7,23.0,325.0,18.6,145.0,73.0,142.0,4.0,0,18
3,74,1,167.6,77.0,103,154.0,78.0,77.0,18.0,95.0,37.1,6.9,5.38,124.0,0.3830,71.2,23.0,324.0,18.5,218.0,85.0,141.0,3.9,0,17
4,74,1,167.6,77.0,103,155.0,61.0,64.0,16.0,92.0,36.7,8.5,4.36,124.0,0.3795,87.5,28.6,326.0,13.6,263.0,81.0,138.0,4.0,0,16


In [112]:
data.to_excel('/content/RNN_before_cleaning.xlsx',index=False)

In [113]:
print(data.columns.tolist())
columns_to_normalize = data.columns.tolist()

['age', 'sex', 'height', 'weight', 'comorbidities', 'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate', 'oxygen_saturation', 'temperature', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count', 'serum_creatinine', 'sodium', 'potassium', 'intubated', 'adjusted_hlos']


In [114]:
#normalisation of columns
from sklearn.preprocessing import MinMaxScaler

df_copy = data.copy()
columns_to_normalize = ['age', 'sex', 'height', 'weight', 'comorbidities', 'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate', 'oxygen_saturation', 'temperature', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count', 'serum_creatinine', 'sodium', 'potassium', 'intubated']
scaler = MinMaxScaler()

df_copy[columns_to_normalize] = scaler.fit_transform(df_copy[columns_to_normalize])

df_copy.head()

,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,serum_creatinine,sodium,potassium,intubated,adjusted_hlos
0,0.679012,1.0,0.583562,0.286795,0.757353,0.363636,0.268041,0.358333,0.207547,0.842105,0.553571,0.198251,0.476667,0.516949,0.507653,0.269618,0.303922,0.607843,0.513514,0.104984,0.071001,0.590909,0.447368,0.0,20
1,0.679012,1.0,0.583562,0.286795,0.757353,0.454545,0.371134,0.308333,0.207547,0.947368,0.535714,0.104956,0.480000,0.508475,0.520408,0.283702,0.289216,0.529412,0.540541,0.106045,0.059025,0.636364,0.342105,0.0,19
2,0.679012,1.0,0.583562,0.286795,0.757353,0.500000,0.474227,0.283333,0.245283,0.868421,0.607143,0.026239,0.525000,0.559322,0.553571,0.259557,0.289216,0.578431,0.554054,0.135737,0.046193,0.590909,0.421053,0.0,18
3,0.679012,1.0,0.583562,0.286795,0.757353,0.590909,0.515464,0.341667,0.207547,0.868421,0.535714,0.154519,0.545000,0.584746,0.581633,0.269618,0.289216,0.568627,0.547297,0.213150,0.056459,0.568182,0.394737,0.0,17
4,0.679012,1.0,0.583562,0.286795,0.757353,0.597403,0.340206,0.233333,0.169811,0.789474,0.464286,0.201166,0.375000,0.584746,0.572704,0.597586,0.563725,0.588235,0.216216,0.260870,0.053037,0.500000,0.421053,0.0,16


In [115]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split,KFold

In [116]:
! pip install scikit-learn

In [118]:
# Features and target
features = df_copy[['age', 'sex', 'height', 'weight', 'comorbidities', 'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate', 'oxygen_saturation', 'temperature', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count', 'serum_creatinine', 'sodium', 'potassium', 'intubated']]
target = df_copy['adjusted_hlos']

In [119]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [120]:
X_train_tensor = torch.FloatTensor(X_train.values).view(-1, 1, features.shape[1]).to(d)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1).to(d)
X_test_tensor = torch.FloatTensor(X_test.values).view(-1, 1, features.shape[1]).to(d)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1).to(d)


In [121]:
import torch.nn as nn
import torch.optim as optim
# Early stopping class
class EarlyStopping:
    def __init__(self, patience=10, delta=0.0001):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [144]:
class LSTMAttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size=128, dropout=0.0):
        super(LSTMAttentionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.attention_layer = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, 1)

    def attention(self, lstm_output):
        attention_scores = self.attention_layer(lstm_output).squeeze(-1)
        attention_weights = torch.softmax(attention_scores, dim=1)
        weighted_output = torch.sum(attention_weights.unsqueeze(-1) * lstm_output, dim=1)
        return weighted_output

    def forward(self, x):
        lstm_output, (h_n, c_n) = self.lstm(x)
        lstm_output = self.dropout(lstm_output)  # Apply dropout after LSTM
        attention_output = self.attention(lstm_output)
        output = self.fc(attention_output)
        return output



In [145]:
from sklearn.metrics import mean_squared_error, r2_score

# Cross-validation function with early stopping and learning rate scheduler
def cross_validation_attention_model(X_train_tensor, y_train_tensor, n_splits=5, batch_size=16, patience=10):
    kf = KFold(n_splits=n_splits)
    mse_scores, r2_scores = [], []

    for train_index, test_index in kf.split(X_train_tensor):
        # Split the data
        X_train, X_test = X_train_tensor[train_index], X_train_tensor[test_index]
        y_train, y_test = y_train_tensor[train_index], y_train_tensor[test_index]

        # Initialize model, criterion, optimizer
        input_size = X_train.shape[2]
        model = LSTMAttentionModel(input_size)
        model.to(d)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.00005)  # Further reduced learning rate
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)
        early_stopping = EarlyStopping(patience=patience)

        num_epochs = 400  # Increased epochs for better learning
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            # Mini-batch training
            num_batches = X_train.shape[0] // batch_size
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = (i + 1) * batch_size
                X_batch = X_train[start_idx:end_idx]
                y_batch = y_train[start_idx:end_idx]

                # Forward pass
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

            # Validate on test set
            model.eval()
            with torch.no_grad():
                val_predictions = model(X_test).cpu().numpy()
                val_actuals = y_test.cpu().numpy()

                val_mse = mean_squared_error(val_actuals, val_predictions)
                scheduler.step(val_mse)

                if early_stopping(val_mse):
                    print(f"Early stopping at epoch {epoch}")
                    break

        # Calculate additional performance metrics
        model.eval()
        with torch.no_grad():
            predictions = model(X_test).cpu().numpy()
            actuals = y_test.cpu().numpy()

            mse = mean_squared_error(actuals, predictions)
            r2 = r2_score(actuals, predictions)
            mse_scores.append(mse)
            r2_scores.append(r2)

    return np.mean(mse_scores), np.std(mse_scores), np.mean(r2_scores), np.std(r2_scores)





In [146]:
mean_mse, std_mse, mean_r2, std_r2 = cross_validation_attention_model(X_train_tensor, y_train_tensor)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [147]:
print(f"Mean MSE: {mean_mse:.4f}, Std MSE: {std_mse:.4f}")
print(f"Mean R²: {mean_r2:.4f}, Std R²: {std_r2:.4f}")

Mean MSE: 130.6280, Std MSE: 12.6336
Mean R²: 0.1156, Std R²: 0.0592


In [148]:
import random
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import KFold

# Define random search function
def random_search_lstm_attention(X_train_tensor, y_train_tensor, param_dist, device, n_iter=10, n_splits=5):
    best_mse, best_r2 = float('inf'), -float('inf')
    best_params = None

    kf = KFold(n_splits=n_splits)

    for i in range(n_iter):
        # Sample hyperparameters randomly
        dropout = random.uniform(param_dist['dropout'][0], param_dist['dropout'][1])
        learning_rate = random.uniform(param_dist['learning_rate'][0], param_dist['learning_rate'][1])
        batch_size = random.choice(param_dist['batch_size'])

        print(f"Iteration {i + 1}/{n_iter} - Dropout: {dropout}, LR: {learning_rate}, Batch Size: {batch_size}")

        mse_scores, r2_scores = [], []

        for train_index, test_index in kf.split(X_train_tensor):
            X_train, X_test = X_train_tensor[train_index], X_train_tensor[test_index]
            y_train, y_test = y_train_tensor[train_index], y_train_tensor[test_index]

            # Initialize model, criterion, optimizer
            input_size = X_train.shape[2]
            model = LSTMAttentionModel(input_size, dropout=dropout)
            model.to(device)
            criterion = torch.nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            # Training loop
            num_epochs = 300
            for epoch in range(num_epochs):
                model.train()
                optimizer.zero_grad()

                num_batches = X_train.shape[0] // batch_size
                for i in range(num_batches):
                    start_idx = i * batch_size
                    end_idx = (i + 1) * batch_size
                    X_batch = X_train[start_idx:end_idx]
                    y_batch = y_train[start_idx:end_idx]

                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)

                    loss.backward()
                    optimizer.step()

            # Evaluate the model on the validation fold
            model.eval()
            with torch.no_grad():
                val_predictions = model(X_test).cpu().numpy()
                val_actuals = y_test.cpu().numpy()

                val_mse = mean_squared_error(val_actuals, val_predictions)
                val_r2 = r2_score(val_actuals, val_predictions)

            mse_scores.append(val_mse)
            r2_scores.append(val_r2)

        # Compute mean scores
        mean_mse = np.mean(mse_scores)
        mean_r2 = np.mean(r2_scores)

        print(f"Mean MSE: {mean_mse:.4f}, Mean R²: {mean_r2:.4f}\n")

        # Save the best parameters
        if mean_mse < best_mse and mean_r2 > best_r2:
            best_mse = mean_mse
            best_r2 = mean_r2
            best_params = {
                'dropout': dropout,
                'learning_rate': learning_rate,
                'batch_size': batch_size
            }

    return best_params, best_mse, best_r2

# Define hyperparameter ranges for random search
param_dist = {
    'dropout': [0.2, 0.5],
    'learning_rate': [1e-5, 1e-3],
    'batch_size': [16, 32, 64],
}







In [149]:
 #Run random search for hyperparameter optimization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
# Run random search for hyperparameter optimization
best_params, best_mse, best_r2 = random_search_lstm_attention(X_train_tensor, y_train_tensor, param_dist, device, n_iter=4)

print("Best Parameters:", best_params)
print("Best MSE:", best_mse)
print("Best R² Score:", best_r2)

Iteration 1/4 - Dropout: 0.22215208166497882, LR: 0.00038332184008690884, Batch Size: 16
Mean MSE: 116.3128, Mean R²: 0.2206

Iteration 2/4 - Dropout: 0.24272408670124418, LR: 0.0008519700731895208, Batch Size: 16


In [ ]:
print("Best Hyperparameters:")
print(best_params)
print(f"Best MSE: {best_mse:.4f}, Best R²: {best_r2:.4f}")